In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as m
from torchsummary import summary

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 1.7.1
GPU found :)


In [ ]:
EMBEDDING_SIZE = 300

In [2]:
# retrieve pretrained model for features extraction

alexnet = m.alexnet(pretrained=True).to(device)
alexnet
#summary(alexnet, (3, 224, 224))

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/jchataigner/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Keep only the feature extraction layers of the model
features = nn.Sequential(*(list(alexnet.children())[0]))
summary(features, (3, 224, 224))

## Build LSTM 

In [ ]:
class LSTMCaptioning(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        self.lstm = nn.LSTM(input_size, hidden_size)
        
    def forward(self, inputs):
         
        # Get hidden states for each t (out) , and latest one (h = (ht, ct))
        out, ht = self.lstm(inputs.view(len(inputs), 1, -1))
        
        # Compute probability distribution over all words for this t
        pt = F.softmax(ht, dim=1)
                           
        return out, ht, pt
        

## Train model

In [ ]:
input_size = (3, 224, 224)
hideen_size = ...

model = LSTMCaptioning(input_size, hidden_size)

num_epoch = 10
loss_function = nn.NLLLoss()

for epoch in range(num_epoch):
    for image, caption in dataset:
        
        # Reset grad
        model.zero_grad()
        
        # Get the input image embedding 
        embedding = features(image)
        targets = ...
        
        # Forward pass
        out, hidden, probs = model(embedding)
        
        # Compute loss and backprop
        loss = loss_function(out, targets)
        loss.backward()
        optimizer.step()
        

In [ ]:
# Save model 
torch.save(model.state_dict(), './models/')